# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.externals import joblib

In [3]:
# load data from database
engine = create_engine('sqlite:///cleaned_data.db')
df = pd.read_sql_table('Messages', engine)
X = df['message']
Y = df.drop(['message', 'id', 'original', 'genre'], axis=1)
print(X.shape)
print(Y.shape)
Y.head()

(26180,)
(26180, 36)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
text = 'Hello Worlds'
text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
words = word_tokenize(text)
#sentences = sent_tokenize(text)
words = [w for w in words if w not in stopwords.words("english")]
#pos_tag(sentence)
#stemmed = [PorterStemmer().stem(w) for w in words]
lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]

In [5]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()   
    tokens = [lemmatizer.lemmatize(tok) for tok in tokens]
    tokens = [lemmatizer.lemmatize(tok, pos='v') for tok in tokens]
    #tokens = [PorterStemmer().stem(tok) for tok in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)
accuracy = (y_pred == y_test).mean()
print(accuracy)

related                   0.810924
request                   0.878915
offer                     0.994652
aid_related               0.749236
medical_help              0.920550
medical_products          0.952063
search_and_rescue         0.970970
security                  0.980328
military                  0.964668
child_alone               1.000000
water                     0.944041
food                      0.915966
shelter                   0.930863
clothing                  0.986822
money                     0.977655
missing_people            0.990260
refugees                  0.964859
death                     0.956455
other_aid                 0.867838
infrastructure_related    0.928762
transport                 0.956264
buildings                 0.951490
electricity               0.981283
tools                     0.994843
hospitals                 0.988350
shops                     0.993316
aid_centers               0.986249
other_infrastructure      0.953400
weather_related     

In [9]:
# column-wise classification report
for i, column in enumerate(y_test.columns):
    print(column)
    #print((y_test[column] == y_pred[:,i]).mean())
    #print(accuracy_score(y_test[column], y_pred[:,i]))
    print(classification_report(y_test[column], y_pred[:,i]))
    #print("Confusion Matrix:\n", confusion_matrix(y_test[column], y_pred[:,i]))
    print('-+'*30)

related
             precision    recall  f1-score   support

          0       0.63      0.50      0.56      1249
          1       0.85      0.91      0.88      3987

avg / total       0.80      0.81      0.80      5236

-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4311
          1       0.82      0.40      0.54       925

avg / total       0.87      0.88      0.86      5236

-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5208
          1       0.00      0.00      0.00        28

avg / total       0.99      0.99      0.99      5236

-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3029
          1       0.76      0.5

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
print(classification_report(y_test, y_pred, target_names=y_test.columns.tolist()))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      3987
               request       0.82      0.40      0.54       925
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.59      0.66      2207
          medical_help       0.57      0.11      0.19       428
      medical_products       0.69      0.07      0.13       261
     search_and_rescue       0.30      0.02      0.04       148
              security       0.00      0.00      0.00       103
              military       0.82      0.05      0.09       192
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.21      0.34       356
                  food       0.80      0.32      0.46       583
               shelter       0.84      0.27      0.41       466
              clothing       0.65      0.15      0.24        74
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa036dde950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [12]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__n_estimators': [50, 100]
    #'clf__estimator__max_depth': [None, 10, 20, 50],
    #'clf__estimator__min_samples_leaf': [1, 2, 4],
    #'clf__estimator__min_samples_split': [2, 5, 7]
    }

#scorer = make_scorer(fbeta_score, beta=0.5); scoring=scorer/'f1_weighted'/'precision_samples'
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=-1)
from workspace_utils import active_session
with active_session():
    cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 1), score=0.2653967344600401, total= 4.9min
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 1) .......


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  6.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 1), score=0.2572697321300673, total= 4.9min
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 1) .......


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 12.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 1), score=0.26758344076779833, total= 5.0min
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 2) .......
[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 2), score=0.27126897737038097, total=13.0min
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 2) .......
[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 2), score=0.2721673112734565, total=13.1min
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 2) .......
[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 2), score=0.2660077352814783, total=13.2min
[CV] clf__estimator__n_estimators=100, vect__ngram_range=(1, 1) ......
[CV]  clf__estimator__n_estimators=100, vect__ngram_range=(1, 1), score=0.26811801775995414, total= 8.3min
[CV] clf__estimator__n_estimators=100, vect__ngram_range=(1, 1) ......
[CV]  clf__estimator__n_estimators=100, vect__ngram_range=(1, 1), score=0.26342930812204557, total= 8.3min
[CV] clf_

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 171.4min finished


In [13]:
print("\nBest Parameters:", cv.best_params_)
#best_clf = cv.best_estimator_


Best Parameters: {'clf__estimator__n_estimators': 50, 'vect__ngram_range': (1, 2)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred = cv.predict(X_test)
accuracy = (y_pred == y_test).mean()
print(accuracy)

related                   0.823911
request                   0.888655
offer                     0.994652
aid_related               0.770244
medical_help              0.921314
medical_products          0.953591
search_and_rescue         0.972880
security                  0.980138
military                  0.963713
child_alone               1.000000
water                     0.954545
food                      0.937548
shelter                   0.934301
clothing                  0.986249
money                     0.977655
missing_people            0.990260
refugees                  0.962949
death                     0.958938
other_aid                 0.870512
infrastructure_related    0.929335
transport                 0.957028
buildings                 0.953782
electricity               0.980901
tools                     0.995034
hospitals                 0.988350
shops                     0.993316
aid_centers               0.986249
other_infrastructure      0.953591
weather_related     

In [15]:
print(classification_report(y_test, y_pred, target_names=y_test.columns.tolist()))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      3987
               request       0.84      0.46      0.59       925
                 offer       0.00      0.00      0.00        28
           aid_related       0.79      0.61      0.69      2207
          medical_help       0.67      0.07      0.13       428
      medical_products       0.85      0.08      0.15       261
     search_and_rescue       0.88      0.05      0.09       148
              security       0.00      0.00      0.00       103
              military       0.75      0.02      0.03       192
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.38      0.53       356
                  food       0.85      0.54      0.66       583
               shelter       0.88      0.30      0.45       466
              clothing       0.58      0.09      0.16        74
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
joblib.dump(cv, 'classifier.pkl')

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.